# Arcgis Pro -- Export 3D Objects -- Mississauga

|             Tool             |                                                    Description                                                     |
|:----------------------------:|:------------------------------------------------------------------------------------------------------------------:|
| Add 3D Formats To Multipatch | Converts a multipatch to a 3D object feature layer by linking the feature class with one or more 3D model formats. |
|      Export 3D Objects       |                          Exports 3D object features to one or more 3D model file formats.                          |



In [ ]:
import arcpy
import os

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

### Add 3D Formats To Multipatch (Data Management)
#### 1. Summary
Converts a multipatch to a 3D object feature layer by linking the feature class with one or more 3D model formats.

In [ ]:
arcpy.env.workspace = r'E:\BuildingWorld\Montreal\cdnndg01_2016_gdb_01_12\CDNNDG01_2016_GDB_01_12\CDNNDG01_2016.gdb'

In [ ]:
print(arcpy.ListFeatureClasses())
print(arcpy.ListFeatureClasses(feature_type="Polygon"))
print(arcpy.ListFeatureClasses(wild_card="Building*"))
# desc = arcpy.Describe('ShapeFile')
# print(desc.shapeType)
print()

In [ ]:
feature_classes = arcpy.ListFeatureClasses()
print(feature_classes)

In [ ]:
workspace = r'E:\BuildingWorld\Montreal\cdnndg01_2016_gdb_01_12\CDNNDG01_2016_GDB_01_12'
for gdb_file in os.listdir(workspace):
    arcpy.env.workspace = os.path.join(workspace, gdb_file)
    if arcpy.env.workspace.endswith('.zip'):
        continue
    feature_classes = arcpy.ListFeatureClasses()
    print(gdb_file)
    print(feature_classes)

In [ ]:
Montreal = r'E:\BuildingWorld\Montreal'
for gdb_folders in os.listdir(Montreal):
    if gdb_folders.endswith('.zip') or gdb_folders == 'arcgis':
        continue
    gdb_folder = os.path.join(Montreal, gdb_folders)
    for folder in os.listdir(gdb_folder):
        workspace = os.path.join(gdb_folder, folder)
        for gdb_file in os.listdir(workspace):
            arcpy.env.workspace = os.path.join(workspace, gdb_file)
            if arcpy.env.workspace.endswith('.zip'):
                continue
            feature_classes = arcpy.ListFeatureClasses()
            print(gdb_file)
            arcpy.management.Add3DFormats(
                in_features="Building_surface",
                multipatch_materials="MULTIPATCH_WITHOUT_MATERIALS",
                formats="FMT3D_OBJ"
            )
            # print(arcpy.env.workspace)
            # print(feature_classes)

```
arcpy.management.Import3DObjects(
    files_and_folders=r"E:\BuildingWorld\Montreal\obj\CDNNDG01_2016.gdb\OID_1",
    updated_features="OID_1",
    update="UPDATE_EXISTING_ADD_NEW",
    translate="0 0",
    elevation=0,
    scale=1,
    rotate=0,
    y_is_up="Y_IS_UP"
)
```

In [ ]:
def export_fc(f, workspace):
    print(f"Exporting: {f}")
    arcpy.env.workspace = workspace
    arcpy.management.Export3DObjects(
        in_features="Building_surface",
        target_folder=os.path.join(r'E:\BuildingWorld\Montreal\obj', f),
        formats="FMT3D_OBJ",
        name_field=None,
        overwrite="OVERWRITE"
    )
    print(f"Finished: {f}")
    arcpy.ClearWorkspaceCache_management()

In [ ]:
Montreal = r'E:\BuildingWorld\Montreal'
tasks = []

# First: gather all valid gdb paths
for gdb_folders in os.listdir(Montreal):
    if gdb_folders.endswith('.zip') or gdb_folders in ['arcgis', 'obj']:
        continue
    gdb_folder = os.path.join(Montreal, gdb_folders)
    for folder in os.listdir(gdb_folder):
        workspace = os.path.join(gdb_folder, folder)
        for gdb_file in os.listdir(workspace):
            if gdb_file.endswith('.zip') or not gdb_file.endswith(".gdb"):
                continue
            gdb_path = os.path.join(workspace, gdb_file)
            tasks.append((gdb_file, gdb_path))

# Then: use tqdm to show progress while submitting to executor
with ThreadPoolExecutor(max_workers=6) as executor:
    futures = [executor.submit(export_fc, f, path) for f, path in tqdm(tasks, desc="Submitting tasks")]

    # Optionally: track progress as tasks complete
    for _ in tqdm(as_completed(futures), total=len(futures), desc="Processing tasks"):
        pass

In [ ]:
# Multipatch_42G
flag = True
for f in feature_classes:
    if f == 'Multipatch_44N':
        flag = False
    if flag:
        continue
    print(f)
    arcpy.management.Export3DObjects(
        in_features=f,
        target_folder=os.path.join("D:\Toronto\obj", f),
        formats="FMT3D_OBJ",
        name_field=None,
        overwrite="OVERWRITE"
    )
